
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

import zipfile
import shutil

In [3]:
#Scrapeo INE datos demográficos por provincia

# lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
chrome_options = webdriver.ChromeOptions()

# establacemos las preferencias que queremos
prefs = {
    "download.default_directory": "C:\\Users\\Víctor\\Desktop\\HACKIO\\Modulo_5\\Archivos_ine",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
    "safebrowsing.enabled": True # protegemos al driver de sitios web peligrosos
}

# configuramos el navegador Chrome con preferencias específicas definidas en el diccionario prefs
chrome_options.add_experimental_option("prefs", prefs)


In [4]:
try:
    url_INE = ("https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990")


    driver=webdriver.Chrome(chrome_options)
    driver.get(url_INE)    
    print("entramos en la web")
    driver.maximize_window()
    sleep(2)
    driver.execute_script("window.scrollTo(0, 300)")
    WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/main/section[2]/div[1]/div[1]/div[1]/ul/li/ul/li[2]/a"))
        ).click()
    print("Hemos clickeado en provincias")
    sleep(3)
    
    driver.execute_script("window.scrollTo(0, 500)")
    WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/main/div[2]/ul/li[4]/ul/li[1]/a"))).click()
    print("Hemos entrado en la población por provincias")
    sleep(5)
    driver.execute_script("window.scrollTo(0, 300)")
    sleep(2)
    cantidad_provincias = (driver.find_elements(By.CSS_SELECTOR, "#cri0" ))
    cantidad_provincias[0].find_elements("css selector", "option")[0].click()
    sleep(2)
    print(f"hemos clickeado provincias")
    cantidad_edad = (driver.find_elements(By.CSS_SELECTOR, "#cri1" ))
    for i in range(0,4):
        cantidad_edad[0].find_elements("css selector", "option")[i].click()

    print(f"hemos clickeado edades")
    type(cantidad_provincias)
    sleep(2)
    cantidad_ext = (driver.find_elements(By.CSS_SELECTOR, "#cri2"))

    for i in range(0,4):
        cantidad_ext[0].find_elements("css selector", "option")[i].click()
    print(f"hemos encontrado valores")
    sleep(2)

    sexo = (driver.find_elements(By.CSS_SELECTOR, "#cri3"))
    for i in range (0,3):
        sexo[0].find_elements("css selector", "option")[i].click()
    print(f"hemos encontrado sexo")
    sleep(2)
    anio = driver.find_elements(By.CSS_SELECTOR,"#periodo")
    for i in range (1,5):
        anio[0].find_elements("css selector", "option")[i].click()
    print(f"hemos encontrado años")
    sleep(2)
    
    driver.execute_script("window.scrollTo(0, 500)")
    WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(("xpath","/html/body/div[1]/main/form/div[5]/input[3]")
    )).click()
    print("Hemos clickeado el botón de selección")

    sleep(5)

    WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable(("xpath","/html/body/div[1]/main/ul/li/div/div/form[2]/button/i"))).click()
    print("vamos a descargar")
    sleep(4)

# Ya estamos en el sub de descarga, vamos a por los datos del iframe:
    iframe = driver.find_element(By.XPATH, "/html/body/div[7]/article/div/div/iframe")
    print("cambio de iframe")
    driver.switch_to.frame(iframe)
    print("estamos en el iframe")
    lista_carga = WebDriverWait(driver,10).until(EC.element_to_be_clickable(("xpath","/html/body/form/ul/li[4]/label"))).click()
    print("hemos encontrado el panel")
    driver.close()


#driver.switch_to.default_content()
except Exception as e:
     print(f"{e}")
    #  driver.close()

entramos en la web
Hemos clickeado en provincias
Hemos entrado en la población por provincias
hemos clickeado provincias
hemos clickeado edades
hemos encontrado valores
hemos encontrado sexo
hemos encontrado años
Hemos clickeado el botón de selección
vamos a descargar
cambio de iframe
estamos en el iframe
hemos encontrado el panel


In [5]:
lista_carga

In [6]:
cantidad_provincias[0].find_elements("xpath","/html/body/form/ul/li[4]/label").text()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00007FF78C853AF5+28005]
	(No symbol) [0x00007FF78C7B83F0]
	(No symbol) [0x00007FF78C65580A]
	(No symbol) [0x00007FF78C62FA85]
	(No symbol) [0x00007FF78C6D2AD7]
	(No symbol) [0x00007FF78C6EB1B1]
	(No symbol) [0x00007FF78C6CB7E3]
	(No symbol) [0x00007FF78C6975C8]
	(No symbol) [0x00007FF78C698731]
	GetHandleVerifier [0x00007FF78CB4646D+3118813]
	GetHandleVerifier [0x00007FF78CB96CC0+3448624]
	GetHandleVerifier [0x00007FF78CB8CF3D+3408301]
	GetHandleVerifier [0x00007FF78C91A44B+841403]
	(No symbol) [0x00007FF78C7C344F]
	(No symbol) [0x00007FF78C7BF4C4]
	(No symbol) [0x00007FF78C7BF65D]
	(No symbol) [0x00007FF78C7AEBB9]
	BaseThreadInitThunk [0x00007FFEBC3EDBE7+23]
	RtlUserThreadStart [0x00007FFEBDE65A4C+44]


In [ ]:
lista_descarga.find_elements

TOTAL ESPAÑA
02 Albacete
03 Alicante/Alacant
04 Almería
01 Araba/Álava
33 Asturias
05 Ávila
06 Badajoz
07 Balears, Illes
08 Barcelona
48 Bizkaia
09 Burgos
10 Cáceres
11 Cádiz
39 Cantabria
12 Castellón/Castelló
13 Ciudad Real
14 Córdoba
15 Coruña, A
16 Cuenca
20 Gipuzkoa
17 Girona
18 Granada
19 Guadalajara
21 Huelva
22 Huesca
23 Jaén
24 León
25 Lleida
27 Lugo
28 Madrid
29 Málaga
30 Murcia
31 Navarra
32 Ourense
34 Palencia
35 Palmas, Las
36 Pontevedra
26 Rioja, La
37 Salamanca
38 Santa Cruz de Tenerife
40 Segovia
41 Sevilla
42 Soria
43 Tarragona
44 Teruel
45 Toledo
46 Valencia/València
47 Valladolid
49 Zamora
50 Zaragoza
51 Ceuta
52 Melilla
